In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
train = pd.read_csv("../input/train.csv")

In [3]:
train.head()

In [4]:
train.describe()

In [5]:
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [6]:
train.loc[train['Age'].isnull()].head()

In [7]:
train.dtypes

In [8]:
embarked_categories = ["C", "Q", "S"]
train['Embarked'] = train['Embarked'].astype('category', categories=embarked_categories)
train['Sex'] = train['Sex'].astype('category')

In [9]:
train['Embarked'] = train['Embarked' ].cat.codes
train['Sex'] = train['Sex' ].cat.codes

In [10]:
train.drop(train.Embarked != -1, inplace = True)

In [11]:
training_data = train[['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

In [12]:
training_data.shape

In [13]:
training_data.describe()

In [14]:
pd.isnull(training_data).any(axis = 0)

In [15]:
# Drop cabin as it has most of the nulls
training_data.drop('Cabin', axis=1, inplace=True)

In [16]:
training_data.dtypes

In [17]:
sb.pairplot(training_data, hue='Survived')

In [18]:
from sklearn import linear_model

In [19]:
training_data.columns.values

In [20]:
test_data = pd.read_csv("../input/test.csv")

In [21]:

test_data['Embarked'] = test_data['Embarked'].astype('category', categories=embarked_categories)
test_data['Embarked'] = test_data['Embarked' ].cat.codes
test_data['Sex'] = test_data['Sex'].astype('category')
test_data['Sex'] = test_data['Sex' ].cat.codes
test_data.isnull().any(axis =0)

In [22]:
 training_data.shape[1]
X_train = training_data.ix[:, 1:training_data.shape[1]]

In [23]:
print(X_train.shape)
y_train = training_data['Survived']

In [24]:
X_test = test_data[X_train.columns.values]
print(X_test.shape)
X_test.isnull().any(axis =0)
X_test[X_test.Fare.isnull()]

In [25]:
fare_mean_by_pclass = X_train[['Pclass', 'Fare']].groupby('Pclass').mean()

In [26]:
X_test.loc[X_test.Fare.isnull(), 'Fare'] = fare_mean_by_pclass.loc[3, 'Fare']
X_test[X_test.Fare.isnull()]

In [27]:
lg = linear_model.LogisticRegression()
lg.fit(X_train, y_train)

In [28]:
y_train_pred = lg.predict(X_train)

In [29]:
from sklearn import metrics

In [30]:
metrics.accuracy_score(y_train, y_train_pred)

In [37]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
clsList = [GaussianNB(), SVC(), DecisionTreeClassifier(), RandomForestClassifier(n_estimators=100)]

In [32]:
from sklearn.model_selection import cross_val_score

In [38]:
for clsfr in clsList:
    scores = cross_val_score(clsfr, X_train, y_train , cv=5, scoring='accuracy')
    print(type(clsfr), scores.mean())

In [34]:
tree = RandomForestClassifier()
tree.fit(X_train, y_train)


In [35]:
y_test_pred = tree.predict(X_test)
test_data['Survived']= y_test_pred
#y_test_pred
#np.savetxt("out.csv", y_test_pred, fmt='%d',header="Survived", comments='')

In [36]:
pred = test_data[['PassengerId', 'Survived']]
pred.to_csv('titanic_pred.csv', index = False)